In [1]:
#!pip install boto3

In [2]:
import boto3
import pandas as pd
import psycopg2
import json

In [3]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('cluster.config'))

In [4]:
config.get("AWS","KEY")

'AKIA6BSGJ45QDZWVV4VGFQ'

In [5]:
KEY = config.get("AWS","KEY")
SECRET = config.get("AWS","SECRET")

DWH_CLUSTER_TYPE = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE = config.get("DWH","DWH_NODE_TYPE")
DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB = config.get("DWH","DWH_DB")
DWH_DB_USER = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT = config.get("DWH","DWH_PORT")
DWH_IAM_ROLE_NAME = config.get("DWH","DWH_IAM_ROLE_NAME")
(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

('jabao', 'yourpass', 'mydb')

In [6]:
pd.DataFrame({"Param":
              ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
              [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
})

,Param,Value
0,DWH_CLUSTER_TYPE,single-node
1,DWH_NUM_NODES,1
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,redshift-1
4,DWH_DB,mydb
5,DWH_DB_USER,jabao
6,DWH_DB_PASSWORD,yourpass
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,redshift-s3-access


In [7]:
ec2 = boto3.resource('ec2',
                    region_name="ap-southeast-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )

In [8]:
s3 = boto3.resource('s3',
                    region_name="ap-southeast-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )

In [9]:
iam = boto3.client('iam',
                    region_name="ap-southeast-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )

In [10]:
redshift = boto3.client('redshift',
                    region_name="ap-southeast-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )

In [11]:
bucket = s3.Bucket("wangjabao")
log_data_files = [filename.key for filename in bucket.objects.filter(Prefix='')]
log_data_files

['allevents_pipe.txt',
 'allusers_pipe.txt',
 'ath-1.png',
 'category_pipe.txt',
 'date2008_pipe.txt',
 'dimDate.csv',
 'listings_pipe.txt',
 'sales_tab.txt',
 'venue_pipe.txt']

In [12]:
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

In [13]:
roleArn

'arn:aws:iam::964643580979:role/redshift-s3-access'

In [14]:
try:
    response = redshift.create_cluster(
    ClusterType=DWH_CLUSTER_TYPE,
    NodeType=DWH_NODE_TYPE,
        
    DBName=DWH_DB,
    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
    MasterUsername=DWH_DB_USER,
    MasterUserPassword=DWH_DB_PASSWORD,
    
    IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

In [19]:
redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

{'ClusterIdentifier': 'redshift-1',
 'NodeType': 'dc2.large',
 'ClusterStatus': 'available',
 'ClusterAvailabilityStatus': 'Available',
 'MasterUsername': 'jabao',
 'DBName': 'mydb',
 'Endpoint': {'Address': 'redshift-1.c5ekcidicr1l.ap-southeast-1.redshift.amazonaws.com',
  'Port': 5439},
 'ClusterCreateTime': datetime.datetime(2023, 5, 29, 4, 26, 58, 768000, tzinfo=tzutc()),
 'AutomatedSnapshotRetentionPeriod': 1,
 'ManualSnapshotRetentionPeriod': -1,
 'ClusterSecurityGroups': [],
 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-09cc01b4c74b60280',
   'Status': 'active'}],
 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
   'ParameterApplyStatus': 'in-sync'}],
 'ClusterSubnetGroupName': 'default',
 'VpcId': 'vpc-0724b65e4901bd7d8',
 'AvailabilityZone': 'ap-southeast-1c',
 'PreferredMaintenanceWindow': 'thu:18:00-thu:18:30',
 'PendingModifiedValues': {},
 'ClusterVersion': '1.0',
 'AllowVersionUpgrade': True,
 'NumberOfNodes': 1,
 'PubliclyAccessible': True,


In [20]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keyToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "VpcId"]
    x = [(k,v) for k,v in props.items() if k in keyToShow]
    return pd.DataFrame(data=x, columns=["Key","Value"])
myClusterProps =  redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3132\1061986636.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Key,Value
0,ClusterIdentifier,redshift-1
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,jabao
4,DBName,mydb
5,Endpoint,"{'Address': 'redshift-1.c5ekcidicr1l.ap-southeast-1.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-0724b65e4901bd7d8


In [21]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
DB_NAME = myClusterProps['DBName']
DB_USER = myClusterProps['MasterUsername']

In [22]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName = defaultSg.group_name,
        CidrIp = '0.0.0.0/0',
        IpProtocol = 'TCP',
        FromPort = int(DWH_PORT),
        ToPort = int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-09cc01b4c74b60280')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


In [36]:
try:
    conn = psycopg2.connect(host=DWH_ENDPOINT, dbname=DB_NAME, user=DB_USER, password=DWH_DB_PASSWORD, port=5439)
except psycopg2.Error as e:
    print("ERROR: Could not make connection to the Postgres database")
    print(e)
    
conn.set_session(autocommit=True)

In [37]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("ERROR: Could not get cursor to the Postgres database")
    print(e)

In [25]:
try:
    cur.execute("""create table users(
    userid integer not null distkey sortkey,
    username char(8),
    firstname varchar(30),
    lastname varchar(30),
    city varchar(30),
    state char(2),
    email varchar(100),
    phone char(14),
    likesports boolean,
    liketheatre boolean,
    likeconcerts boolean,
    likejazz boolean,
    likeclassical boolean,
    likeopera boolean,
    likerock boolean,
    likevegas boolean,
    likebroadway boolean,
    likemusicals boolean);""")
except psycopg2.Error as e:
    print("ERROR: Issue creating table")
    print(e)

In [26]:
try:
    cur.execute("""create table venue(
    venueid smallint not null distkey sortkey,
    venuename varchar(100),
    venuecity varchar(30),
    venuestate char(2),
    venueseats integer);""")
except psycopg2.Error as e:
    print("ERROR: Issue creating table")
    print(e)

In [27]:
try:
    cur.execute("""create table category(
    catid smallint not null distkey sortkey,
    catgroup varchar(10),
    catname varchar(10),
    catdesc varchar(50));""")
except psycopg2.Error as e:
    print("ERROR: Issue creating table")
    print(e)

In [28]:
try:
    cur.execute("""create table date(
    dateid smallint not null distkey sortkey,
    caldate date not null,
    day character(3) not null,
    week smallint not null,
    month character(5) not null,
    qtr character(3) not null,
    year smallint not null,
    holiday boolean default('N'));""")
except psycopg2.Error as e:
    print("ERROR: Issue creating table")
    print(e)

In [29]:
try:
    cur.execute("""create table event(
    eventid integer not null distkey,
    venueid smallint not null,
    catid smallint not null,
    dateid smallint not null sortkey,
    eventname varchar(200),
    starttime timestamp);""")
except psycopg2.Error as e:
    print("ERROR: Issue creating table")
    print(e)

In [30]:
try:
    cur.execute("""create table listing(
    listid integer not null distkey,
    sellerid integer not null,
    eventid integer not null,
    dateid smallint not null sortkey,
    numtickets smallint not null,
    priceperticket decimal(8,2),
    listtime timestamp);""")
except psycopg2.Error as e:
    print("ERROR: Issue creating table")
    print(e)

In [31]:
try:
    cur.execute("""
    copy users from 's3://wangjabao/allusers_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::964643580979:role/redshift-s3-access'
    delimiter '|'
    region 'ap-southeast-1'
    
    """)
except psycopg2.Error as e:
    print("ERROR: Issue creating table")
    print(e)

In [32]:
try:
    cur.execute("""
    select * from users;
    
    """)
except psycopg2.Error as e:
    print("ERROR: Issue creating table")
    print(e)

In [33]:
row = cur.fetchone()
print(row)

(2, 'PGL08LJI', 'Vladimir', 'Humphrey', 'Murfreesboro', 'SK', 'Suspendisse.tristique@nonnisiAenean.edu', '(783) 492-1886', None, None, None, True, True, None, None, True, False, True)


In [34]:
try:
    conn.close()
except psycopg2.Error as e:
    print(e)

In [35]:
#redshift.delete_cluster(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True)